### Construcción de un Sistema RAG con LangChain y ChromaDB
#### Introducción
La Generación Aumentada por Recuperación (RAG) es una técnica poderosa que combina las capacidades de los modelos de lenguaje grandes con la recuperación de conocimiento externo. Este cuaderno te guiará a través de la construcción de un sistema RAG completo usando:

- LangChain: Un framework para desarrollar aplicaciones impulsadas por modelos de lenguaje
- ChromaDB: Una base de datos vectorial de código abierto para almacenar y recuperar embeddings
- OpenAI: Para embeddings y modelo de lenguaje (puedes sustituir con otros proveedores)

In [16]:
# Cargar variables de entorno para API keys
import os
from dotenv import load_dotenv
load_dotenv()  # Carga las variables de entorno desde el archivo .env

True

In [17]:
# Importaciones de langchain para procesamiento de documentos
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Para dividir documentos en chunks
from langchain_community.document_loaders import TextLoader  # Para cargar documentos de texto
from langchain_openai import OpenAIEmbeddings  # Para generar embeddings con OpenAI
from langchain.schema import Document  # Clase base para documentos

# Importaciones de vectorstores (almacenes vectoriales)
from langchain_community.vectorstores import Chroma  # Base de datos vectorial ChromaDB

# Importaciones de utilidades
import numpy as np  # Para operaciones matemáticas con arrays
from typing import List  # Para anotaciones de tipos

In [18]:
# Resumen de la Arquitectura RAG
print("""
Arquitectura RAG (Generación Aumentada por Recuperación):

1. Carga de Documentos: Cargar documentos desde varias fuentes
2. División de Documentos: Dividir documentos en chunks más pequeños
3. Generación de Embeddings: Convertir chunks en representaciones vectoriales
4. Almacenamiento Vectorial: Almacenar embeddings en ChromaDB
5. Procesamiento de Consultas: Convertir consulta del usuario a embedding
6. Búsqueda por Similitud: Encontrar chunks relevantes del almacén vectorial
7. Aumento de Contexto: Combinar chunks recuperados con la consulta
8. Generación de Respuesta: El LLM genera respuesta usando el contexto

Beneficios de RAG:
- Reduce las alucinaciones
- Proporciona información actualizada
- Permite citar fuentes
- Funciona con conocimiento específico del dominio
""")


Arquitectura RAG (Generación Aumentada por Recuperación):

1. Carga de Documentos: Cargar documentos desde varias fuentes
2. División de Documentos: Dividir documentos en chunks más pequeños
3. Generación de Embeddings: Convertir chunks en representaciones vectoriales
4. Almacenamiento Vectorial: Almacenar embeddings en ChromaDB
5. Procesamiento de Consultas: Convertir consulta del usuario a embedding
6. Búsqueda por Similitud: Encontrar chunks relevantes del almacén vectorial
7. Aumento de Contexto: Combinar chunks recuperados con la consulta
8. Generación de Respuesta: El LLM genera respuesta usando el contexto

Beneficios de RAG:
- Reduce las alucinaciones
- Proporciona información actualizada
- Permite citar fuentes
- Funciona con conocimiento específico del dominio



### 1. Datos de Muestra

In [19]:
# Crear documentos de muestra con información sobre Machine Learning
sample_docs = [
    """
    Fundamentos del Aprendizaje Automático (Machine Learning)
    
    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite 
    a los sistemas aprender y mejorar a partir de la experiencia sin ser programados 
    explícitamente. Hay tres tipos principales de aprendizaje automático: aprendizaje 
    supervisado, aprendizaje no supervisado y aprendizaje por refuerzo. El aprendizaje 
    supervisado utiliza datos etiquetados para entrenar modelos, mientras que el 
    aprendizaje no supervisado encuentra patrones en datos no etiquetados. El aprendizaje 
    por refuerzo aprende a través de la interacción con un entorno utilizando recompensas 
    y penalizaciones.
    """,
    
    """
    Aprendizaje Profundo y Redes Neuronales
    
    El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes 
    neuronales artificiales. Estas redes están inspiradas en el cerebro humano y consisten 
    en capas de nodos interconectados. El aprendizaje profundo ha revolucionado campos 
    como la visión por computadora, el procesamiento de lenguaje natural y el reconocimiento 
    de voz. Las Redes Neuronales Convolucionales (CNN) son particularmente efectivas para 
    el procesamiento de imágenes, mientras que las Redes Neuronales Recurrentes (RNN) y 
    los Transformers destacan en el procesamiento de datos secuenciales.
    """,
    
    """
    Procesamiento de Lenguaje Natural (PLN)
    
    El PLN es un campo de la IA que se enfoca en la interacción entre computadoras y 
    lenguaje humano. Las tareas clave en PLN incluyen clasificación de texto, 
    reconocimiento de entidades nombradas, análisis de sentimientos, traducción automática 
    y respuesta a preguntas. El PLN moderno depende en gran medida de arquitecturas 
    transformer como BERT, GPT y T5. Estos modelos utilizan mecanismos de atención para 
    entender el contexto y las relaciones entre palabras en el texto.
    """
]

sample_docs  # Mostrar los documentos creados

['\n    Fundamentos del Aprendizaje Automático (Machine Learning)\n\n    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite \n    a los sistemas aprender y mejorar a partir de la experiencia sin ser programados \n    explícitamente. Hay tres tipos principales de aprendizaje automático: aprendizaje \n    supervisado, aprendizaje no supervisado y aprendizaje por refuerzo. El aprendizaje \n    supervisado utiliza datos etiquetados para entrenar modelos, mientras que el \n    aprendizaje no supervisado encuentra patrones en datos no etiquetados. El aprendizaje \n    por refuerzo aprende a través de la interacción con un entorno utilizando recompensas \n    y penalizaciones.\n    ',
 '\n    Aprendizaje Profundo y Redes Neuronales\n\n    El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes \n    neuronales artificiales. Estas redes están inspiradas en el cerebro humano y consisten \n    en capas de nodos interconectados. El a

In [23]:
# Guardar documentos de muestra en archivos temporales para poder cargarlos después
import tempfile
temp_dir = tempfile.mkdtemp()  # Crear directorio temporal

# Escribir cada documento en un archivo separado
for i, doc in enumerate(sample_docs):
    with open(f"{temp_dir}/doc_{i}.txt", "w", encoding='utf-8') as f:
        f.write(doc)

print(f"Documentos de muestra creados en: {temp_dir}")

Documentos de muestra creados en: C:\Users\raule\AppData\Local\Temp\tmp7yg3sy23


In [24]:
# Alternativa: guardar documentos en el directorio actual (opcional)
import tempfile
#temp_dir = tempfile.mkdtemp()

# Escribir documentos en archivos individuales
for i, doc in enumerate(sample_docs):
    with open(f"doc_{i}.txt", "w", encoding='utf-8') as f:
        f.write(doc)

### 2. Carga de Documentos

In [25]:
# Cargar documentos desde el directorio usando LangChain
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# Crear un cargador de documentos que busque archivos .txt en el directorio temporal
loader = DirectoryLoader(
    temp_dir,  # Directorio donde están los archivos
    glob="*.txt",  # Patrón para buscar solo archivos .txt
    loader_cls=TextLoader,  # Clase para cargar archivos de texto
    loader_kwargs={'encoding': 'utf-8'}  # Especificar codificación UTF-8
)

# Cargar todos los documentos encontrados
documents = loader.load()

print(f"Cargados {len(documents)} documentos")

# Verificar si hay documentos antes de intentar acceder
if len(documents) > 0:
    print(f"\nVista previa del primer documento:")
    print(documents[0].page_content[:200] + "...")
else:
    print("\nNo se encontraron documentos en el directorio.")
    print(f"Verificando directorio: {temp_dir}")
    import os
    if os.path.exists(temp_dir):
        files = os.listdir(temp_dir)
        print(f"Archivos en el directorio: {files}")
    else:
        print("El directorio temporal no existe.")

Cargados 3 documentos

Vista previa del primer documento:

    Fundamentos del Aprendizaje Automático (Machine Learning)

    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite 
    a los sistemas aprender y mejorar a parti...


In [26]:
# Mostrar todos los documentos cargados con sus metadatos
documents

[Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_0.txt'}, page_content='\n    Fundamentos del Aprendizaje Automático (Machine Learning)\n\n    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite \n    a los sistemas aprender y mejorar a partir de la experiencia sin ser programados \n    explícitamente. Hay tres tipos principales de aprendizaje automático: aprendizaje \n    supervisado, aprendizaje no supervisado y aprendizaje por refuerzo. El aprendizaje \n    supervisado utiliza datos etiquetados para entrenar modelos, mientras que el \n    aprendizaje no supervisado encuentra patrones en datos no etiquetados. El aprendizaje \n    por refuerzo aprende a través de la interacción con un entorno utilizando recompensas \n    y penalizaciones.\n    '),
 Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_1.txt'}, page_content='\n    Aprendizaje Profundo y Redes Neuronales\n\n    El a

### 3. División de Documentos (Document Splitting)

In [27]:
# Inicializar el divisor de texto para crear chunks más pequeños
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Tamaño máximo de cada chunk en caracteres
    chunk_overlap=50,  # Superposición entre chunks para mantener contexto
    length_function=len,  # Función para medir la longitud
    separators=[" "]  # Jerarquía de separadores (espacios en este caso)
)

# Dividir todos los documentos en chunks más pequeños
chunks = text_splitter.split_documents(documents)

print(f"Creados {len(chunks)} chunks a partir de {len(documents)} documentos")
print(f"\nEjemplo de chunk:")
print(f"Contenido: {chunks[0].page_content[:150]}...")
print(f"Metadatos: {chunks[0].metadata}")

Creados 6 chunks a partir de 3 documentos

Ejemplo de chunk:
Contenido: Fundamentos del Aprendizaje Automático (Machine Learning)

    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite 
...
Metadatos: {'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_0.txt'}


In [28]:
# Mostrar todos los chunks creados para inspección
chunks

[Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_0.txt'}, page_content='Fundamentos del Aprendizaje Automático (Machine Learning)\n\n    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite \n    a los sistemas aprender y mejorar a partir de la experiencia sin ser programados \n    explícitamente. Hay tres tipos principales de aprendizaje automático: aprendizaje \n    supervisado, aprendizaje no supervisado y aprendizaje por refuerzo. El aprendizaje \n    supervisado utiliza datos etiquetados para entrenar modelos, mientras que el'),
 Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_0.txt'}, page_content='para entrenar modelos, mientras que el \n    aprendizaje no supervisado encuentra patrones en datos no etiquetados. El aprendizaje \n    por refuerzo aprende a través de la interacción con un entorno utilizando recompensas \n    y penalizaciones.'),
 Document(metadata={'source

### 4. Modelos de Embedding

In [29]:
# Configurar la API key de OpenAI desde las variables de entorno
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [30]:
# Probar el modelo de embeddings con un texto de ejemplo
sample_text = "El Aprendizaje Automático es fascinante"
embeddings = OpenAIEmbeddings()  # Inicializar el modelo de embeddings de OpenAI
embeddings  # Mostrar configuración del modelo

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000021533328B60>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000021533C649E0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [31]:
# Generar embedding para el texto de muestra
vector = embeddings.embed_query(sample_text)  # Convierte el texto en vector numérico
vector  # Mostrar el vector resultante (será una lista de números)

[-0.026229841634631157,
 0.008753953501582146,
 0.027715515345335007,
 -0.01675226166844368,
 -0.019762033596634865,
 0.015727659687399864,
 -0.011008080095052719,
 -0.002105239313095808,
 -0.005612904205918312,
 -0.029559802263975143,
 0.01067508477717638,
 0.03493896871805191,
 -0.0023822023067623377,
 0.010162782855331898,
 0.0029105134308338165,
 -0.004015163518488407,
 0.020620139315724373,
 0.007889444939792156,
 -0.00010020899208029732,
 0.003184274537488818,
 -0.02053048647940159,
 0.017533522099256516,
 -0.007345124147832394,
 -0.03709063678979874,
 -0.02630668692290783,
 0.00011306656233500689,
 0.006711150985211134,
 -0.04003636911511421,
 0.003893492044880986,
 -0.023604296147823334,
 0.03542565554380417,
 -0.011392306536436081,
 -0.0028672879561781883,
 -0.03442666679620743,
 -0.007063358090817928,
 -0.030866170302033424,
 -0.012372083030641079,
 -0.02078663744032383,
 -0.003336364170536399,
 0.01200706884264946,
 0.021145248785614967,
 0.01978764869272709,
 -0.00815199874

### 5. Inicializar ChromaDB y Almacenar los Chunks en Representación Vectorial

In [32]:
# Verificar los chunks antes de almacenarlos
chunks

[Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_0.txt'}, page_content='Fundamentos del Aprendizaje Automático (Machine Learning)\n\n    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite \n    a los sistemas aprender y mejorar a partir de la experiencia sin ser programados \n    explícitamente. Hay tres tipos principales de aprendizaje automático: aprendizaje \n    supervisado, aprendizaje no supervisado y aprendizaje por refuerzo. El aprendizaje \n    supervisado utiliza datos etiquetados para entrenar modelos, mientras que el'),
 Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_0.txt'}, page_content='para entrenar modelos, mientras que el \n    aprendizaje no supervisado encuentra patrones en datos no etiquetados. El aprendizaje \n    por refuerzo aprende a través de la interacción con un entorno utilizando recompensas \n    y penalizaciones.'),
 Document(metadata={'source

In [33]:
# Crear la base de datos vectorial ChromaDB
persist_directory = "./chroma_db"  # Directorio donde se guardará la base de datos

# Inicializar ChromaDB con embeddings de OpenAI
vectorstore = Chroma.from_documents(
    documents=chunks,  # Los chunks de texto a almacenar
    embedding=OpenAIEmbeddings(),  # Modelo para generar embeddings
    persist_directory=persist_directory,  # Directorio de persistencia
    collection_name="rag_collection"  # Nombre de la colección
)

print(f"Base de datos vectorial creada con {vectorstore._collection.count()} vectores")
print(f"Persistida en: {persist_directory}")

Base de datos vectorial creada con 6 vectores
Persistida en: ./chroma_db


### 6. Probar Búsqueda por Similitud

In [34]:
# Realizar búsqueda por similitud - encontrar documentos similares a la consulta
query = "¿Cuáles son los tipos de aprendizaje automático?"

# Buscar los 3 documentos más similares a la consulta
similar_docs = vectorstore.similarity_search(query, k=3)
similar_docs  # Mostrar los documentos encontrados

[Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_0.txt'}, page_content='Fundamentos del Aprendizaje Automático (Machine Learning)\n\n    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite \n    a los sistemas aprender y mejorar a partir de la experiencia sin ser programados \n    explícitamente. Hay tres tipos principales de aprendizaje automático: aprendizaje \n    supervisado, aprendizaje no supervisado y aprendizaje por refuerzo. El aprendizaje \n    supervisado utiliza datos etiquetados para entrenar modelos, mientras que el'),
 Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_1.txt'}, page_content='Aprendizaje Profundo y Redes Neuronales\n\n    El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes \n    neuronales artificiales. Estas redes están inspiradas en el cerebro humano y consisten \n    en capas de nodos interconectados. El aprendiz

In [35]:
# Probar otra consulta sobre Procesamiento de Lenguaje Natural
query = "¿Qué es el PLN?"

# Buscar documentos relacionados con PLN
similar_docs = vectorstore.similarity_search(query, k=3)
similar_docs

[Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_2.txt'}, page_content='Procesamiento de Lenguaje Natural (PLN)\n\n    El PLN es un campo de la IA que se enfoca en la interacción entre computadoras y \n    lenguaje humano. Las tareas clave en PLN incluyen clasificación de texto, \n    reconocimiento de entidades nombradas, análisis de sentimientos, traducción automática \n    y respuesta a preguntas. El PLN moderno depende en gran medida de arquitecturas \n    transformer como BERT, GPT y T5. Estos modelos utilizan mecanismos de atención para \n    entender el'),
 Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_1.txt'}, page_content='Aprendizaje Profundo y Redes Neuronales\n\n    El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes \n    neuronales artificiales. Estas redes están inspiradas en el cerebro humano y consisten \n    en capas de nodos interconectados. El aprendizaje

In [36]:
# Probar consulta sobre Aprendizaje Profundo
query = "¿Qué es el Aprendizaje Profundo?"

# Buscar documentos relacionados con Deep Learning
similar_docs = vectorstore.similarity_search(query, k=3)
similar_docs

[Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_1.txt'}, page_content='Aprendizaje Profundo y Redes Neuronales\n\n    El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes \n    neuronales artificiales. Estas redes están inspiradas en el cerebro humano y consisten \n    en capas de nodos interconectados. El aprendizaje profundo ha revolucionado campos \n    como la visión por computadora, el procesamiento de lenguaje natural y el reconocimiento \n    de voz. Las Redes Neuronales Convolucionales (CNN) son particularmente efectivas para'),
 Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_0.txt'}, page_content='Fundamentos del Aprendizaje Automático (Machine Learning)\n\n    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite \n    a los sistemas aprender y mejorar a partir de la experiencia sin ser programados \n    explícitamente. Hay tres tipos

In [37]:
# Mostrar los resultados de búsqueda de manera más legible
print(f"Consulta: {query}")
print(f"\nLos {len(similar_docs)} chunks más similares:")

for i, doc in enumerate(similar_docs):
    print(f"\n--- Chunk {i+1} ---")
    print(doc.page_content[:200] + "...")  # Mostrar solo los primeros 200 caracteres
    print(f"Fuente: {doc.metadata.get('source', 'Desconocida')}")

Consulta: ¿Qué es el Aprendizaje Profundo?

Los 3 chunks más similares:

--- Chunk 1 ---
Aprendizaje Profundo y Redes Neuronales

    El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes 
    neuronales artificiales. Estas redes están inspiradas en el cereb...
Fuente: C:\Users\raule\AppData\Local\Temp\tmp7yg3sy23\doc_1.txt

--- Chunk 2 ---
Fundamentos del Aprendizaje Automático (Machine Learning)

    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite 
    a los sistemas aprender y mejorar a partir de ...
Fuente: C:\Users\raule\AppData\Local\Temp\tmp7yg3sy23\doc_0.txt

--- Chunk 3 ---
para entrenar modelos, mientras que el 
    aprendizaje no supervisado encuentra patrones en datos no etiquetados. El aprendizaje 
    por refuerzo aprende a través de la interacción con un entorno ut...
Fuente: C:\Users\raule\AppData\Local\Temp\tmp7yg3sy23\doc_0.txt


### 7. Búsqueda por Similitud Avanzada con Puntuaciones

In [39]:
# Búsqueda con puntuaciones de similitud para evaluar qué tan relevantes son los resultados
results_scores = vectorstore.similarity_search_with_score(query, k=3)
results_scores  # Mostrar documentos con sus puntuaciones de similitud

[(Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_1.txt'}, page_content='Aprendizaje Profundo y Redes Neuronales\n\n    El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes \n    neuronales artificiales. Estas redes están inspiradas en el cerebro humano y consisten \n    en capas de nodos interconectados. El aprendizaje profundo ha revolucionado campos \n    como la visión por computadora, el procesamiento de lenguaje natural y el reconocimiento \n    de voz. Las Redes Neuronales Convolucionales (CNN) son particularmente efectivas para'),
  0.23319245874881744),
 (Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_0.txt'}, page_content='Fundamentos del Aprendizaje Automático (Machine Learning)\n\n    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite \n    a los sistemas aprender y mejorar a partir de la experiencia sin ser programados \n    expl

#### Entendiendo las Puntuaciones de Similitud

La puntuación de similitud representa qué tan relacionado está un chunk de documento con tu consulta. La puntuación depende de la métrica de distancia utilizada:

**ChromaDB por defecto: Usa distancia L2 (distancia Euclidiana)**
- Puntuaciones más bajas = MÁS similar (más cerca en el espacio vectorial)
- Puntuación de 0 = vectores idénticos
- Rango típico: 0 a 2 (pero puede ser mayor)

**Similitud coseno (si se configura):**
- Puntuaciones más altas = MÁS similar
- Rango: -1 a 1 (1 siendo idéntico)

### 8. Inicializar LLM, Cadena RAG, Plantilla de Prompt y Consultar el Sistema RAG

In [40]:
# Inicializar el modelo de lenguaje de OpenAI para generar respuestas
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo"  # Usar GPT-3.5 Turbo para respuestas rápidas y económicas
)

In [41]:
# Probar el LLM con una consulta simple
test_response = llm.invoke("¿Qué son los Modelos de Lenguaje Grandes?")
test_response  # Mostrar la respuesta del modelo

AIMessage(content='Los Modelos de Lenguaje Grandes, también conocidos como Modelos de Lenguaje de Gran Escala, son modelos de inteligencia artificial diseñados para procesar y comprender el lenguaje humano a gran escala. Estos modelos utilizan técnicas de aprendizaje automático y procesamiento del lenguaje natural para analizar, generar y entender texto de manera más avanzada que los modelos tradicionales.\n\nEstos modelos utilizan grandes cantidades de datos de texto para entrenarse y aprender patrones lingüísticos y contextuales con el fin de mejorar su capacidad para entender y generar texto de manera más precisa y coherente. Algunos ejemplos de Modelos de Lenguaje Grandes son GPT-3 de OpenAI y BERT de Google. Estos modelos han demostrado ser muy útiles en una amplia gama de aplicaciones, como la traducción automática, la generación de texto y el análisis de sentimientos, entre otros.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 210,

In [42]:
# Método alternativo para inicializar modelos de chat
from langchain.chat_models.base import init_chat_model

llm = init_chat_model("openai:gpt-3.5-turbo")  # Inicializar usando string de configuración
# llm = init_chat_model("groq:")  # Alternativa con Groq (comentado)
llm  # Mostrar configuración del modelo

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021536BDB320>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021536BECC80>, root_client=<openai.OpenAI object at 0x0000021536BD8770>, root_async_client=<openai.AsyncOpenAI object at 0x0000021536BDB920>, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [43]:
# Probar el modelo con otra consulta
llm.invoke("¿Qué es la Inteligencia Artificial?")

AIMessage(content='La Inteligencia Artificial es una rama de la tecnología que se enfoca en el diseño y desarrollo de sistemas y programas informáticos capaces de realizar tareas que normalmente requieren de la inteligencia humana, como el aprendizaje, la resolución de problemas, el reconocimiento de patrones y la toma de decisiones. Estos sistemas utilizan algoritmos y técnicas avanzadas de procesamiento de datos para simular la inteligencia humana en máquinas y lograr que estas puedan aprender y adaptarse a diferentes situaciones de manera autónoma.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 124, 'prompt_tokens': 16, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-COkHWT6HBcQQhI

### 9. Cadena RAG Moderna

In [44]:
# Importar componentes necesarios para crear la cadena RAG moderna
from langchain.chains import create_retrieval_chain  # Para crear cadena de recuperación
from langchain_core.prompts import ChatPromptTemplate  # Para plantillas de prompts
from langchain.chains.combine_documents import create_stuff_documents_chain  # Para combinar documentos

In [45]:
# Convertir el almacén vectorial en un recuperador (retriever)
retriever = vectorstore.as_retriever(
    search_kwarg={"k": 3}  # Recuperar los 3 chunks más relevantes por defecto
)
retriever  # Mostrar configuración del recuperador

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000215333270B0>, search_kwargs={})

In [46]:
# Crear una plantilla de prompt para el sistema RAG
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """Eres un asistente para tareas de responder preguntas. 
Usa los siguientes fragmentos de contexto recuperado para responder la pregunta. 
Si no conoces la respuesta, simplemente di que no lo sabes. 
Usa máximo tres oraciones y mantén la respuesta concisa.

Contexto: {context}"""

# Crear la plantilla de prompt con mensaje de sistema y humano
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),  # Instrucciones del sistema
    ("human", "{input}")  # Input del usuario
])

In [47]:
# Mostrar la configuración del prompt
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Eres un asistente para tareas de responder preguntas. \nUsa los siguientes fragmentos de contexto recuperado para responder la pregunta. \nSi no conoces la respuesta, simplemente di que no lo sabes. \nUsa máximo tres oraciones y mantén la respuesta concisa.\n\nContexto: {context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

##### ¿Qué es create_stuff_documents_chain?
create_stuff_documents_chain crea una cadena que "rellena" (inserta) todos los documentos recuperados en un solo prompt y lo envía al LLM. Se llama "stuff" porque literalmente rellena todos los documentos en la ventana de contexto de una vez.

In [48]:
# Crear una cadena de documentos que combina el LLM con el prompt
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain  # Mostrar la cadena creada

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Eres un asistente para tareas de responder preguntas. \nUsa los siguientes fragmentos de contexto recuperado para responder la pregunta. \nSi no conoces la respuesta, simplemente di que no lo sabes. \nUsa máximo tres oraciones y mantén la respuesta concisa.\n\nContexto: {context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021536BDB320>, async_cli

Esta cadena:

- Toma los documentos recuperados
- Los "rellena" en el marcador de posición {context} del prompt
- Envía el prompt completo al LLM
- Devuelve la respuesta del LLM

#### ¿Qué es create_retrieval_chain?
create_retrieval_chain es una función que combina un recuperador (que busca documentos relevantes) con una cadena de documentos (que procesa esos documentos con un LLM) para crear un pipeline RAG completo.

In [49]:
# Crear la cadena RAG final combinando recuperador y cadena de documentos
from langchain.chains import create_retrieval_chain
rag_chain = create_retrieval_chain(retriever, document_chain)
rag_chain  # Mostrar la cadena RAG completa

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000215333270B0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Eres un asistente para tareas de responder preguntas. \nUsa los siguientes fragmentos de contexto recuperado para responder la pregunta.

In [50]:
# Probar la cadena RAG con una consulta
response = rag_chain.invoke({"input": "¿Qué es el Aprendizaje Profundo?"})

In [51]:
# Mostrar la respuesta completa (incluye input, contexto y respuesta)
response

{'input': '¿Qué es el Aprendizaje Profundo?',
 'context': [Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_1.txt'}, page_content='Aprendizaje Profundo y Redes Neuronales\n\n    El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes \n    neuronales artificiales. Estas redes están inspiradas en el cerebro humano y consisten \n    en capas de nodos interconectados. El aprendizaje profundo ha revolucionado campos \n    como la visión por computadora, el procesamiento de lenguaje natural y el reconocimiento \n    de voz. Las Redes Neuronales Convolucionales (CNN) son particularmente efectivas para'),
  Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_0.txt'}, page_content='Fundamentos del Aprendizaje Automático (Machine Learning)\n\n    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite \n    a los sistemas aprender y mejorar a partir de la experienc

In [52]:
# Extraer solo la respuesta del sistema RAG
response['answer']

'El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes neuronales artificiales con múltiples capas de nodos interconectados. Se ha usado para revolucionar campos como la visión por computadora, el procesamiento de lenguaje natural y el reconocimiento de voz.'

In [53]:
# Función para consultar el sistema RAG moderno de manera más organizada
def query_rag_modern(question):
    """
    Función que consulta el sistema RAG y muestra resultados de forma legible
    """
    print(f"Pregunta: {question}")
    print("-" * 50)
    
    # Usar la cadena RAG para obtener respuesta
    result = rag_chain.invoke({"input": question})
    
    print(f"Respuesta: {result['answer']}")
    print("\nContexto Recuperado:")
    for i, doc in enumerate(result['context']):
        print(f"\n--- Fuente {i+1} ---")
        print(doc.page_content[:200] + "...")
    
    return result

# Preguntas de prueba para evaluar el sistema
test_questions = [
    "¿Cuáles son los tres tipos de aprendizaje automático?",
    "¿Qué es el aprendizaje profundo y cómo se relaciona con las redes neuronales?",
    "¿Para qué se usan mejor las CNN?"
]

# Probar cada pregunta
for question in test_questions:
    result = query_rag_modern(question)
    print("\n" + "="*80 + "\n")

Pregunta: ¿Cuáles son los tres tipos de aprendizaje automático?
--------------------------------------------------
Respuesta: Los tres tipos principales de aprendizaje automático son: aprendizaje supervisado, aprendizaje no supervisado y aprendizaje por refuerzo.

Contexto Recuperado:

--- Fuente 1 ---
Fundamentos del Aprendizaje Automático (Machine Learning)

    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite 
    a los sistemas aprender y mejorar a partir de ...

--- Fuente 2 ---
Aprendizaje Profundo y Redes Neuronales

    El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes 
    neuronales artificiales. Estas redes están inspiradas en el cereb...

--- Fuente 3 ---
para entrenar modelos, mientras que el 
    aprendizaje no supervisado encuentra patrones en datos no etiquetados. El aprendizaje 
    por refuerzo aprende a través de la interacción con un entorno ut...

--- Fuente 4 ---
(CNN) son particularmente efec

### 10. Crear Cadena RAG Alternativa - Usando LCEL (Lenguaje de Expresión de LangChain)

In [54]:
# Enfoque más flexible usando LCEL (LangChain Expression Language)
from langchain_core.output_parsers import StrOutputParser  # Para convertir salida a string
from langchain_core.runnables import RunnablePassthrough, RunnableParallel  # Para operaciones avanzadas

In [55]:
# Crear un prompt personalizado para LCEL
custom_prompt = ChatPromptTemplate.from_template("""Usa el siguiente contexto para responder la pregunta. 
Si no conoces la respuesta basándote en el contexto, di que no lo sabes.
Proporciona detalles específicos del contexto para respaldar tu respuesta.

Contexto:
{context}

Pregunta: {question}

Respuesta:""")
custom_prompt  # Mostrar la plantilla de prompt personalizada

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Usa el siguiente contexto para responder la pregunta. \nSi no conoces la respuesta basándote en el contexto, di que no lo sabes.\nProporciona detalles específicos del contexto para respaldar tu respuesta.\n\nContexto:\n{context}\n\nPregunta: {question}\n\nRespuesta:'), additional_kwargs={})])

In [56]:
# Verificar el recuperador
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000215333270B0>, search_kwargs={})

In [57]:
# Función para formatear los documentos de salida para el prompt
def format_docs(docs):
    """
    Combina múltiples documentos en un string único separado por líneas
    """
    return "\n\n".join(doc.page_content for doc in docs)

In [58]:
# Construir la cadena usando LCEL (sintaxis más expresiva)
rag_chain_lcel = (
    { 
        "context": retriever | format_docs,  # Recuperar documentos y formatearlos
        "question": RunnablePassthrough()  # Pasar la pregunta directamente
     }
    | custom_prompt  # Aplicar el prompt personalizado
    | llm  # Enviar al modelo de lenguaje
    | StrOutputParser()  # Convertir salida a string
)

rag_chain_lcel  # Mostrar la cadena LCEL construida

{
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000215333270B0>, search_kwargs={})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Usa el siguiente contexto para responder la pregunta. \nSi no conoces la respuesta basándote en el contexto, di que no lo sabes.\nProporciona detalles específicos del contexto para respaldar tu respuesta.\n\nContexto:\n{context}\n\nPregunta: {question}\n\nRespuesta:'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021536BDB320>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0

In [59]:
# Probar la cadena LCEL con una consulta
response = rag_chain_lcel.invoke("¿Qué es el Aprendizaje Profundo?")
response  # Mostrar solo la respuesta (string)

'El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes neuronales artificiales. Estas redes están inspiradas en el cerebro humano y consisten en capas de nodos interconectados. El aprendizaje profundo ha revolucionado campos como la visión por computadora, el procesamiento de lenguaje natural y el reconocimiento de voz.'

In [60]:
# Obtener documentos relevantes directamente del recuperador (método alternativo)
retriever.get_relevant_documents("¿Qué es el Aprendizaje Profundo?")

C:\Users\raule\AppData\Local\Temp\ipykernel_20252\1689932662.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("¿Qué es el Aprendizaje Profundo?")


[Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_1.txt'}, page_content='Aprendizaje Profundo y Redes Neuronales\n\n    El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes \n    neuronales artificiales. Estas redes están inspiradas en el cerebro humano y consisten \n    en capas de nodos interconectados. El aprendizaje profundo ha revolucionado campos \n    como la visión por computadora, el procesamiento de lenguaje natural y el reconocimiento \n    de voz. Las Redes Neuronales Convolucionales (CNN) son particularmente efectivas para'),
 Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_0.txt'}, page_content='Fundamentos del Aprendizaje Automático (Machine Learning)\n\n    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite \n    a los sistemas aprender y mejorar a partir de la experiencia sin ser programados \n    explícitamente. Hay tres tipos

In [61]:
# Función para consultar usando el enfoque LCEL - Versión corregida
def query_rag_lcel(question):
    """
    Consulta el sistema RAG usando la cadena LCEL
    """
    print(f"Pregunta: {question}")
    print("-" * 50)
    
    # Método 1: Pasar string directamente (cuando se usa RunnablePassthrough)
    answer = rag_chain_lcel.invoke(question)
    print(f"Respuesta: {answer}")
    
    # Obtener documentos fuente por separado si es necesario
    docs = retriever.get_relevant_documents(question)
    print("\nDocumentos Fuente:")
    for i, doc in enumerate(docs):
        print(f"\n--- Fuente {i+1} ---")
        print(doc.page_content[:200] + "...")

In [62]:
# Probar la cadena LCEL
print("Probando Cadena LCEL:")
query_rag_lcel("¿Cuáles son los conceptos clave en el aprendizaje por refuerzo?")

Probando Cadena LCEL:
Pregunta: ¿Cuáles son los conceptos clave en el aprendizaje por refuerzo?
--------------------------------------------------
Respuesta: Los conceptos clave en el aprendizaje por refuerzo son la interacción con un entorno utilizando recompensas y penalizaciones.

Documentos Fuente:

--- Fuente 1 ---
para entrenar modelos, mientras que el 
    aprendizaje no supervisado encuentra patrones en datos no etiquetados. El aprendizaje 
    por refuerzo aprende a través de la interacción con un entorno ut...

--- Fuente 2 ---
mecanismos de atención para 
    entender el contexto y las relaciones entre palabras en el texto....

--- Fuente 3 ---
Fundamentos del Aprendizaje Automático (Machine Learning)

    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite 
    a los sistemas aprender y mejorar a partir de ...

--- Fuente 4 ---
Aprendizaje Profundo y Redes Neuronales

    El aprendizaje profundo es un subconjunto del aprendizaje automático 

In [65]:
query_rag_lcel("Que es el  aprendizaje automatico?")

Pregunta: Que es el  aprendizaje automatico?
--------------------------------------------------
Respuesta: El aprendizaje automático es un subconjunto de la inteligencia artificial que permite a los sistemas aprender y mejorar a partir de la experiencia sin ser programados explícitamente. Se basa en el uso de datos para entrenar modelos y hay tres tipos principales de aprendizaje automático: supervisado, no supervisado y por refuerzo.

Documentos Fuente:

--- Fuente 1 ---
Fundamentos del Aprendizaje Automático (Machine Learning)

    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite 
    a los sistemas aprender y mejorar a partir de ...

--- Fuente 2 ---
Aprendizaje Profundo y Redes Neuronales

    El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes 
    neuronales artificiales. Estas redes están inspiradas en el cereb...

--- Fuente 3 ---
para entrenar modelos, mientras que el 
    aprendizaje no supervisado encuentr

In [66]:
query_rag_lcel("Que es el  aprendizaje profundo?")

Pregunta: Que es el  aprendizaje profundo?
--------------------------------------------------
Respuesta: El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes neuronales artificiales. Estas redes están inspiradas en el cerebro humano y consisten en capas de nodos interconectados. El aprendizaje profundo ha revolucionado campos como la visión por computadora, el procesamiento de lenguaje natural y el reconocimiento de voz.

Documentos Fuente:

--- Fuente 1 ---
Aprendizaje Profundo y Redes Neuronales

    El aprendizaje profundo es un subconjunto del aprendizaje automático basado en redes 
    neuronales artificiales. Estas redes están inspiradas en el cereb...

--- Fuente 2 ---
Fundamentos del Aprendizaje Automático (Machine Learning)

    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite 
    a los sistemas aprender y mejorar a partir de ...

--- Fuente 3 ---
para entrenar modelos, mientras que el 
    aprendizaje no su

### 11. Agregar Nuevos Documentos al Almacén Vectorial Existente

In [67]:
# Verificar el almacén vectorial actual
vectorstore

In [68]:
# Agregar nuevos documentos al almacén vectorial existente
new_document = """
Aprendizaje por Refuerzo en Detalle

El aprendizaje por refuerzo (RL) es un tipo de aprendizaje automático donde un agente 
aprende a tomar decisiones interactuando con un entorno. El agente recibe recompensas 
o penalizaciones basadas en sus acciones y aprende a maximizar la recompensa acumulada 
a lo largo del tiempo. Los conceptos clave en RL incluyen: estados, acciones, recompensas, 
políticas y funciones de valor. Los algoritmos populares de RL incluyen Q-learning, 
Deep Q-Networks (DQN), métodos de Gradiente de Política y métodos Actor-Crítico. 
El RL se ha aplicado exitosamente en juegos (como AlphaGo), robótica y sistemas autónomos.
"""

In [69]:
new_document

'\nAprendizaje por Refuerzo en Detalle\n\nEl aprendizaje por refuerzo (RL) es un tipo de aprendizaje automático donde un agente \naprende a tomar decisiones interactuando con un entorno. El agente recibe recompensas \no penalizaciones basadas en sus acciones y aprende a maximizar la recompensa acumulada \na lo largo del tiempo. Los conceptos clave en RL incluyen: estados, acciones, recompensas, \npolíticas y funciones de valor. Los algoritmos populares de RL incluyen Q-learning, \nDeep Q-Networks (DQN), métodos de Gradiente de Política y métodos Actor-Crítico. \nEl RL se ha aplicado exitosamente en juegos (como AlphaGo), robótica y sistemas autónomos.\n'

In [70]:
chunks

[Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_0.txt'}, page_content='Fundamentos del Aprendizaje Automático (Machine Learning)\n\n    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite \n    a los sistemas aprender y mejorar a partir de la experiencia sin ser programados \n    explícitamente. Hay tres tipos principales de aprendizaje automático: aprendizaje \n    supervisado, aprendizaje no supervisado y aprendizaje por refuerzo. El aprendizaje \n    supervisado utiliza datos etiquetados para entrenar modelos, mientras que el'),
 Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_0.txt'}, page_content='para entrenar modelos, mientras que el \n    aprendizaje no supervisado encuentra patrones en datos no etiquetados. El aprendizaje \n    por refuerzo aprende a través de la interacción con un entorno utilizando recompensas \n    y penalizaciones.'),
 Document(metadata={'source

In [71]:
# Crear un nuevo documento con metadatos personalizados
new_doc = Document(
    page_content=new_document,
    metadata={"source": "adicion_manual", "topic": "aprendizaje_por_refuerzo"}  # Metadatos en español
)

In [72]:
new_doc

Document(metadata={'source': 'adicion_manual', 'topic': 'aprendizaje_por_refuerzo'}, page_content='\nAprendizaje por Refuerzo en Detalle\n\nEl aprendizaje por refuerzo (RL) es un tipo de aprendizaje automático donde un agente \naprende a tomar decisiones interactuando con un entorno. El agente recibe recompensas \no penalizaciones basadas en sus acciones y aprende a maximizar la recompensa acumulada \na lo largo del tiempo. Los conceptos clave en RL incluyen: estados, acciones, recompensas, \npolíticas y funciones de valor. Los algoritmos populares de RL incluyen Q-learning, \nDeep Q-Networks (DQN), métodos de Gradiente de Política y métodos Actor-Crítico. \nEl RL se ha aplicado exitosamente en juegos (como AlphaGo), robótica y sistemas autónomos.\n')

In [73]:
# Dividir el nuevo documento en chunks
new_chunks = text_splitter.split_documents([new_doc])
new_chunks  # Mostrar los chunks creados

[Document(metadata={'source': 'adicion_manual', 'topic': 'aprendizaje_por_refuerzo'}, page_content='Aprendizaje por Refuerzo en Detalle\n\nEl aprendizaje por refuerzo (RL) es un tipo de aprendizaje automático donde un agente \naprende a tomar decisiones interactuando con un entorno. El agente recibe recompensas \no penalizaciones basadas en sus acciones y aprende a maximizar la recompensa acumulada \na lo largo del tiempo. Los conceptos clave en RL incluyen: estados, acciones, recompensas, \npolíticas y funciones de valor. Los algoritmos populares de RL incluyen Q-learning, \nDeep Q-Networks (DQN),'),
 Document(metadata={'source': 'adicion_manual', 'topic': 'aprendizaje_por_refuerzo'}, page_content='RL incluyen Q-learning, \nDeep Q-Networks (DQN), métodos de Gradiente de Política y métodos Actor-Crítico. \nEl RL se ha aplicado exitosamente en juegos (como AlphaGo), robótica y sistemas autónomos.')]

In [74]:
# Agregar nuevos documentos al almacén vectorial existente
vectorstore.add_documents(new_chunks)  # Agrega los chunks y devuelve IDs únicos

['2937d6e1-8b33-4cc8-a999-25913a4bfb84',
 '816f00b6-2292-424c-a817-6d2612b4f4dd']

In [75]:
# Verificar que se agregaron los nuevos chunks
print(f"Se agregaron {len(new_chunks)} nuevos chunks al almacén vectorial")
print(f"Total de vectores ahora: {vectorstore._collection.count()}")

Se agregaron 2 nuevos chunks al almacén vectorial
Total de vectores ahora: 8


In [76]:
# Consultar con el almacén vectorial actualizado (que incluye el nuevo documento)
new_question = "¿Cuáles son los conceptos clave en el aprendizaje por refuerzo?"
result = query_rag_lcel(new_question)
result

Pregunta: ¿Cuáles son los conceptos clave en el aprendizaje por refuerzo?
--------------------------------------------------
Respuesta: Los conceptos clave en el aprendizaje por refuerzo incluyen: estados, acciones, recompensas, políticas y funciones de valor.

Documentos Fuente:

--- Fuente 1 ---
Aprendizaje por Refuerzo en Detalle

El aprendizaje por refuerzo (RL) es un tipo de aprendizaje automático donde un agente 
aprende a tomar decisiones interactuando con un entorno. El agente recibe re...

--- Fuente 2 ---
para entrenar modelos, mientras que el 
    aprendizaje no supervisado encuentra patrones en datos no etiquetados. El aprendizaje 
    por refuerzo aprende a través de la interacción con un entorno ut...

--- Fuente 3 ---
mecanismos de atención para 
    entender el contexto y las relaciones entre palabras en el texto....

--- Fuente 4 ---
Fundamentos del Aprendizaje Automático (Machine Learning)

    El aprendizaje automático es un subconjunto de la inteligencia artificial 

### 12. Técnicas RAG Avanzadas - Memoria Conversacional

**Entendiendo la Memoria Conversacional en RAG**

La memoria conversacional permite que los sistemas RAG mantengan contexto a través de múltiples interacciones. Esto es crucial para:

- Preguntas de seguimiento que referencian respuestas anteriores
- Resolución de pronombres (ej: "esto", "ellos", "eso")
- Consultas dependientes del contexto que se basan en discusión previa
- Flujo de diálogo natural donde los usuarios no repiten contexto

**Desafío Clave:**
El RAG tradicional recupera documentos basándose solo en la consulta actual, perdiendo contexto importante de la conversación. Por ejemplo:

Usuario: "Háblame sobre Python"
Bot: explica el lenguaje de programación Python
Usuario: "¿Cuáles son sus principales librerías?" ← "sus" se refiere a Python, pero el recuperador no lo sabe

**Solución:**
El enfoque moderno usa un proceso de dos pasos:

1. **Reformulación de Consulta:** Transforma preguntas dependientes del contexto en consultas independientes
2. **Recuperación Consciente del Contexto:** Usa la consulta reformulada para buscar documentos relevantes

**Componentes Clave:**
- create_history_aware_retriever: Hace que el recuperador entienda el contexto de la conversación
- MessagesPlaceholder: Marcador de posición para el historial de chat en prompts
- HumanMessage/AIMessage: Tipos de mensaje estructurados para historial de conversación

In [77]:
# Importar componentes para memoria conversacional
from langchain.chains import create_history_aware_retriever  # Para recuperador consciente del historial
from langchain_core.prompts import MessagesPlaceholder  # Para incluir historial en prompts
from langchain_core.messages import HumanMessage, AIMessage  # Para estructurar mensajes

In [78]:
# Crear un prompt que incluye el historial de chat
contextualize_q_system_prompt = """Dado un historial de chat y la última pregunta del usuario 
que podría referenciar contexto en el historial de chat, formula una pregunta independiente 
que pueda entenderse sin el historial de chat. NO respondas la pregunta, 
solo reformúlala si es necesario; de lo contrario, devuélvela tal como está."""

# Crear plantilla de prompt con marcador de posición para historial
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),  # Instrucciones del sistema
    MessagesPlaceholder("chat_history"),  # Aquí se insertará el historial
    ("human", "{input}"),  # Entrada del usuario actual
])

In [79]:
# Crear recuperador consciente del historial
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt  # LLM, recuperador base y prompt de contextualización
)
history_aware_retriever  # Mostrar el recuperador mejorado

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000215333270B0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIM

In [81]:
# Create a new document chain with history
qa_system_prompt = """Eres asistente en tareas de preguntas y respuestas.
Usa los siguientes fragmentos de contexto recuperados para responder la pregunta.
Si no sabes la respuesta, simplemente di que no la sabes.
Usa un máximo de tres oraciones y mantén la respuesta concisa..

Context: {context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create conversational RAG chain
conversational_rag_chain = create_retrieval_chain(
    history_aware_retriever, 
    question_answer_chain
)
print("Se creó una cadena RAG conversacional!")

Se creó una cadena RAG conversacional!


In [82]:
chat_history=[]
# First question
result1 = conversational_rag_chain.invoke({
    "chat_history": chat_history,
    "input": "que es el aprendizaje automatico?"
})
print(f"Q: que es el aprendizaje automatico?")
print(f"A: {result1['answer']}")

Q: que es el aprendizaje automatico?
A: El aprendizaje automático es un subconjunto de la inteligencia artificial que permite a los sistemas aprender y mejorar a partir de la experiencia sin ser programados explícitamente, utilizando principalmente datos.


In [83]:
chat_history.extend([
    HumanMessage(content="que es el aprendizaje automatico"),
    AIMessage(content=result1['answer'])
])

In [84]:
chat_history

[HumanMessage(content='que es el aprendizaje automatico', additional_kwargs={}, response_metadata={}),
 AIMessage(content='El aprendizaje automático es un subconjunto de la inteligencia artificial que permite a los sistemas aprender y mejorar a partir de la experiencia sin ser programados explícitamente, utilizando principalmente datos.', additional_kwargs={}, response_metadata={})]

In [85]:
## Follow up question
# Follow-up question
result2 = conversational_rag_chain.invoke({
    "chat_history": chat_history,
    "input": "¿Cuáles son sus principales tipos?"  # Refers to ML from previous question
})
result2

{'chat_history': [HumanMessage(content='que es el aprendizaje automatico', additional_kwargs={}, response_metadata={}),
  AIMessage(content='El aprendizaje automático es un subconjunto de la inteligencia artificial que permite a los sistemas aprender y mejorar a partir de la experiencia sin ser programados explícitamente, utilizando principalmente datos.', additional_kwargs={}, response_metadata={})],
 'input': '¿Cuáles son sus principales tipos?',
 'context': [Document(metadata={'source': 'C:\\Users\\raule\\AppData\\Local\\Temp\\tmp7yg3sy23\\doc_0.txt'}, page_content='Fundamentos del Aprendizaje Automático (Machine Learning)\n\n    El aprendizaje automático es un subconjunto de la inteligencia artificial que permite \n    a los sistemas aprender y mejorar a partir de la experiencia sin ser programados \n    explícitamente. Hay tres tipos principales de aprendizaje automático: aprendizaje \n    supervisado, aprendizaje no supervisado y aprendizaje por refuerzo. El aprendizaje \n    sup

In [86]:
result2['answer']

'Los tres tipos principales de aprendizaje automático son: aprendizaje supervisado, aprendizaje no supervisado y aprendizaje por refuerzo.'

### Uso de GROQ LLM
 

In [97]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D96AA93110>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D96AA92FD0>, root_client=<openai.OpenAI object at 0x000001D96AB19810>, root_async_client=<openai.AsyncOpenAI object at 0x000001D96AB196E0>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [87]:
load_dotenv()

True

In [88]:
os.getenv("GROQ_API_KEY")

''

In [89]:
from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model

In [90]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [91]:
llm=ChatGroq(model="gemma2-9b-it",api_key=os.getenv("GROQ_API_KEY"))
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021537273020>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000215382B3C20>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr(''))

In [92]:
llm=init_chat_model(model="groq:gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000215382B3DD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000215382B4290>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr(''))